<a href="https://colab.research.google.com/github/minalin0530/Miniproject3/blob/main/roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
import tensorflow as tf
import tensorflow_hub as hub

In [3]:
from transformers import RobertaTokenizer, TFRobertaModel
from transformers import BertTokenizer, TFBertModel
from transformers import XLNetTokenizer, TFXLNetModel

from tensorflow.keras.optimizers import Adam
import os

In [4]:
import pandas as pd
transcript = pd.read_csv("ASR.csv")

In [5]:
models = {'roberta':(RobertaTokenizer,'roberta-large',TFRobertaModel),
          'bert':(BertTokenizer, 'bert-large-uncased', TFBertModel),
          'xlnet':(XLNetTokenizer, 'xlnet-large-cased', TFXLNetModel)}

In [6]:
tokenizer, model_type, model_name = models['roberta']

In [7]:
def make_inputs(tokenizer, model_type, serie, max_len= 128):

    tokenizer = tokenizer.from_pretrained(model_type, lowercase=True )
    tokenized_data = [tokenizer.encode_plus(text, max_length=max_len,
                                            padding='max_length',
                                            add_special_tokens=True,
                                            truncation = True) for text in serie]


    input_ids = np.array([text['input_ids'] for text in tokenized_data])
    attention_mask = np.array([text['attention_mask'] for text in tokenized_data])

    return input_ids, attention_mask

In [8]:
import numpy as np
input_ids_train, attention_mask_train = \
make_inputs(tokenizer, model_type, transcript["transcription"], max_len= 128)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

In [9]:
##### TPU or no TPU
def init_model(model_name, model_type, num_labels, Tpu = 'on', max_len = 128):
# ------------------------------------------------ with TPU --------------------------------------------------------------#
    if Tpu == 'on':
        # a few lines of code to get our tpu started and our data distributed on it
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        # print("All devices: ", tf.config.list_logical_devices('TPU'))

        strategy = tf.distribute.experimental.TPUStrategy(resolver)
        with strategy.scope():

            model_ = model_name.from_pretrained(model_type)
            # inputs
            input_ids = tf.keras.Input(shape = (max_len, ), dtype = 'int32')
            attention_masks = tf.keras.Input(shape = (max_len,), dtype = 'int32')

            outputs = model_([input_ids, attention_masks])

            if 'xlnet' in model_type:
                # cls is the last token in xlnet tokenization
                outputs = outputs[0]
                cls_output = tf.squeeze(outputs[:, -1:, :], axis=1)
            else:
                cls_output = outputs[1]

            final_output = tf.keras.layers.Dense(num_labels, activation = 'softmax')(cls_output)
            model = tf.keras.Model(inputs = [input_ids, attention_masks], outputs = final_output)
            model.compile(optimizer = Adam(lr = 1e-5), loss = 'categorical_crossentropy',
                        metrics = ['accuracy'])
# ------------------------------------------------ without TPU --------------------------------------------------------------#
    else:
        model_ = model_name.from_pretrained(model_type)
        # inputs
        input_ids = tf.keras.Input(shape = (max_len, ), dtype = 'int32')
        attention_masks = tf.keras.Input(shape = (max_len,), dtype = 'int32')

        outputs = model_([input_ids, attention_masks])

        if 'xlnet' in model_type:
            # cls is the last token in xlnet tokenization
            outputs = outputs[0]
            cls_output = tf.squeeze(outputs[:, -1:, :], axis=1)
        else:
            cls_output = outputs[1]


        final_output = tf.keras.layers.Dense(num_labels, activation = 'softmax')(cls_output)

        model = tf.keras.Model(inputs = [input_ids, attention_masks], outputs = final_output)

        model.compile(optimizer = Adam(lr = 1e-5), loss = 'categorical_crossentropy',
                    metrics = ['accuracy'])
    return model

In [10]:
model = init_model(model_name, model_type, num_labels = 2, Tpu = 'off', max_len = 128)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [11]:
train_y = tf.keras.utils.to_categorical(transcript["label"], num_classes=2)
del transcript

In [ ]:
model.fit([input_ids_train, attention_mask_train], train_y,
          validation_split=0.4, epochs = 10, batch_size = 1,
          shuffle = True)

Epoch 1/10
99/99 [==============================] - 1302s 13s/step - loss: 2.7566 - accuracy: 0.5152 - val_loss: 0.7332 - val_accuracy: 0.3881
Epoch 2/10
99/99 [==============================] - 1213s 12s/step - loss: 0.8948 - accuracy: 0.4848 - val_loss: 0.9240 - val_accuracy: 0.3881
Epoch 3/10
99/99 [==============================] - 1198s 12s/step - loss: 0.8649 - accuracy: 0.4747 - val_loss: 1.1620 - val_accuracy: 0.3881
Epoch 4/10
99/99 [==============================] - 1196s 12s/step - loss: 0.8750 - accuracy: 0.5051 - val_loss: 0.6708 - val_accuracy: 0.6119
Epoch 5/10
99/99 [==============================] - 1196s 12s/step - loss: 0.8162 - accuracy: 0.5556 - val_loss: 0.6724 - val_accuracy: 0.6119
Epoch 6/10
99/99 [==============================] - 1195s 12s/step - loss: 0.8173 - accuracy: 0.5455 - val_loss: 0.9437 - val_accuracy: 0.3881
Epoch 7/10
17/99 [====>.........................] - ETA: 15:02 - loss: 1.0121 - accuracy: 0.5294